In [1]:
import pandas as pd
import numpy as np
import json
import pandas as pd
import re
from rdkit import Chem
from rdkit.Chem import AllChem
from collections import Counter,defaultdict
from ast import literal_eval
import cobra
from cobra import Metabolite,Reaction,Gene
from tqdm import tqdm
import random
import pickle
from multiprocessing import Pool

import sys
# sys.path.append('../Script/')
from common import *
# from check_newinfo_getyeast8U import *

In [2]:
### input
yeast870_path =  '../../Data/model/yeast-GEM.yml'
cut_off = 0.3
num = 50
num_process = 50
cut_off_path = f'../../Results/not_lipid/top{num}_{cut_off}_re/'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'

### output
rxndb_total_info_to_model_del_reaction_path = '../../Results/not_lipid/top50_0.3_re/rxndb_total_info_to_model_del_reaction_top50_0.3.csv'
error_reaction_path = '../../Results/not_lipid/top50_0.3_re/error_reaction_random/'
error_reaction_all_path = '../../Results/not_lipid/top50_0.3_re/error_reaction_all.pickle'
yeast8U_del_path = '../../Data/model/yeast8U_del_re.yml'
yeast8U_mod_path = '../../Data/model/yeast8U_mod_re.yml'
biomass_up_reaction_path = '../../Results/not_lipid/top50_0.3_re/biomass_up_reaction_top50_0.3.pickle'

In [3]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_path)
rxndb_total_info_to_model.shape

(7889, 8)

In [7]:
get_yeast8U(metabolites_info_to_GEM_path,rxndb_total_info_to_model_path,yeast8_reaction_in_rxndb_json,yeast870_path,yeast8U_mod_path)

(4151, 4)
(7889, 8)
(7889, 8)
(6537, 8)


100%|██████████| 6537/6537 [00:47<00:00, 138.45it/s]


In [8]:
yeast8U_mod = cobra.io.load_yaml_model(yeast8U_mod_path)
yeast8U_mod.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.9952,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.07041,0,0.00%
s_0925,r_1861,0.009193,0,0.00%
s_1324,r_2005,0.06967,0,0.00%
s_1374,r_2020,0.0005053,0,0.00%
s_1438,r_2049,0.0005526,0,0.00%
s_1468,r_2060,0.002784,0,0.00%
s_4200,r_4593,0.0001796,0,0.00%
s_4201,r_4594,9.173E-05,0,0.00%


In [9]:
yeast8 = cobra.io.load_yaml_model(yeast870_path)


In [10]:
yeast8U_mod_path

'../../Data/model/yeast8U_mod_re.yml'

In [11]:
yeast8U_mod

Name,None
Memory address,7f54ecf003d0
Number of metabolites,3224
Number of reactions,10668
Number of genes,2078
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


# find reactions that affect biomass

In [12]:
reaction_id = []
for i in yeast8U_mod.reactions:
    if 'rxn' in i.id:
        i.bounds = (0,0)
        reaction_id.append(i.id)

biomass_up_reaction = []
for i in tqdm(reaction_id,total=len(reaction_id)):
    with yeast8U_mod:
        yeast8U_mod.reactions.get_by_id(i).bounds = (0,1000)
        yeast8U_mod.solver = 'gurobi'
        yeast8U_mod.optimize()
        if yeast8U_mod.reactions.get_by_id('r_2111').flux > 0.09:
            biomass_up_reaction.append(i)
            print(i,'error_reaction')
        else: 
            pass
        
biomass_up_reaction

  1%|          | 50/6537 [00:21<45:16,  2.39it/s] 

rxn26590 error_reaction


  5%|▍         | 308/6537 [01:58<26:45,  3.88it/s] 

rxn22365 error_reaction


 10%|▉         | 647/6537 [03:15<27:47,  3.53it/s]

...

rxn950 error_reaction


 98%|█████████▊| 6418/6537 [25:14<00:26,  4.56it/s]

rxn1099 error_reaction


100%|██████████| 6537/6537 [25:40<00:00,  4.24it/s]


['rxn26590',
 'rxn22365',
 'rxn22821',
 'rxn14698',
 'rxn14699',
 'rxn15074',
 'rxn15223',
 'rxn17720',
 'rxn38229',
 'rxn13585',
 'rxn13609',
 'rxn13633',
 'rxn13645',
 'rxn13657',
 'rxn13669',
 'rxn13681',
 'rxn13693',
 'rxn13705',
 'rxn13717',
 'rxn16396',
 'rxn49044',
 'rxn49474',
 'rxn49475',
 'rxn47715',
 'rxn47725',
 'rxn47740',
 'rxn53495',
 'rxn55573',
 'rxn49522',
 'rxn66844',
 'rxn66848',
 'rxn66854',
 'rxn67272',
 'rxn9667',
 'rxn9668',
 'rxn7509',
 'rxn6111',
 'rxn6112',
 'rxn4566',
 'rxn4569',
 'rxn4570',
 'rxn4726',
 'rxn5754',
 'rxn3363',
 'rxn3366',
 'rxn3543',
 'rxn3544',
 'rxn3545',
 'rxn3546',
 'rxn3547',
 'rxn4260',
 'rxn1650',
 'rxn1652',
 'rxn1656',
 'rxn1861',
 'rxn2777',
 'rxn95',
 'rxn377',
 'rxn950',
 'rxn1099']

In [13]:
# Save the list as a pickle file
with open(biomass_up_reaction_path, 'wb') as f:
    pickle.dump(biomass_up_reaction, f)

In [14]:

# # Load the pickle file
with open(biomass_up_reaction_path, 'rb') as f:
    biomass_up_reaction = pickle.load(f)

# View the loaded data
print(biomass_up_reaction)

['rxn26590', 'rxn22365', 'rxn22821', 'rxn14698', 'rxn14699', 'rxn15074', 'rxn15223', 'rxn17720', 'rxn38229', 'rxn13585', 'rxn13609', 'rxn13633', 'rxn13645', 'rxn13657', 'rxn13669', 'rxn13681', 'rxn13693', 'rxn13705', 'rxn13717', 'rxn16396', 'rxn49044', 'rxn49474', 'rxn49475', 'rxn47715', 'rxn47725', 'rxn47740', 'rxn53495', 'rxn55573', 'rxn49522', 'rxn66844', 'rxn66848', 'rxn66854', 'rxn67272', 'rxn9667', 'rxn9668', 'rxn7509', 'rxn6111', 'rxn6112', 'rxn4566', 'rxn4569', 'rxn4570', 'rxn4726', 'rxn5754', 'rxn3363', 'rxn3366', 'rxn3543', 'rxn3544', 'rxn3545', 'rxn3546', 'rxn3547', 'rxn4260', 'rxn1650', 'rxn1652', 'rxn1656', 'rxn1861', 'rxn2777', 'rxn95', 'rxn377', 'rxn950', 'rxn1099']


In [15]:
atp_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0434' in tmp and tmp['s_0434']>0:
            atp_reaction.append(reaction.id)
            
print(len(atp_reaction))
print(atp_reaction)

103
['rxn28200', 'rxn21496', 'rxn21497', 'rxn21498', 'rxn21592', 'rxn24686', 'rxn24920', 'rxn23184', 'rxn32939', 'rxn39117', 'rxn16466', 'rxn16728', 'rxn44269', 'rxn44270', 'rxn44271', 'rxn45307', 'rxn46595', 'rxn46596', 'rxn31322', 'rxn31339', 'rxn31356', 'rxn31373', 'rxn31395', 'rxn31404', 'rxn31423', 'rxn31462', 'rxn31497', 'rxn31654', 'rxn31655', 'rxn31656', 'rxn31658', 'rxn31659', 'rxn31660', 'rxn31791', 'rxn47319', 'rxn47825', 'rxn47826', 'rxn52685', 'rxn58872', 'rxn58873', 'rxn58878', 'rxn58880', 'rxn58882', 'rxn58883', 'rxn58886', 'rxn63373', 'rxn64593', 'rxn64594', 'rxn60705', 'rxn60855', 'rxn60963', 'rxn61150', 'rxn61179', 'rxn61338', 'rxn67231', 'rxn60424', 'rxn60676', 'rxn68760', 'rxn68765', 'rxn9378', 'rxn9379', 'rxn9383', 'rxn9534', 'rxn9535', 'rxn9947', 'rxn9948', 'rxn70470', 'rxn7958', 'rxn7960', 'rxn7962', 'rxn8388', 'rxn9982', 'rxn9983', 'rxn9985', 'rxn9986', 'rxn9987', 'rxn9994', 'rxn9995', 'rxn9997', 'rxn9999', 'rxn10011', 'rxn10013', 'rxn10015', 'rxn10025', 'rxn100

In [16]:
atp_sim_name = ['UDP','UMP','UTP',
                'dUDP','dUMP','dUTP',
                'CDP','CMP','CTP',
                'dCDP','dCMP','dCTP',
                'TDP','TMP','TTP',
                'dTDP','dTMP','dTTP',
                'GDP','GMP','GTP',
                'dGDP','dGMP','dGTP',
                'IDP','IMP','ITP',
                'dIDP','dIMP','dITP',
                ]
atp_sim_list = []
for met in atp_sim_name:
    for i in yeast8.metabolites:
        if i.compartment == 'c':
            if i.name == met:
                print(i.id,i.name,met)
                atp_sim_list.append(i.id)

atp_sim_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if any(x in tmp for x in atp_sim_list):
            atp_sim_reaction.append(reaction.id)
print(len(atp_sim_reaction))
print(atp_sim_reaction)

s_1538 UDP UDP
s_1545 UMP UMP
s_1559 UTP UTP
s_0652 dUDP dUDP
s_0654 dUMP dUMP
s_0656 dUTP dUTP
s_0467 CDP CDP
s_0526 CMP CMP
s_0539 CTP CTP
s_0587 dCDP dCDP
s_0589 dCMP dCMP
s_0590 dCTP dCTP
s_1475 TDP TDP
s_1497 TMP TMP
s_0647 dTDP dTDP
s_0649 dTMP dTMP
s_0650 dTTP dTTP
s_0739 GDP GDP
s_0782 GMP GMP
s_0785 GTP GTP
s_0613 dGDP dGDP
s_0615 dGMP dGMP
s_0617 dGTP dGTP
s_0846 IDP IDP
s_0849 IMP IMP
s_0950 ITP ITP
s_0618 dIDP dIDP
s_0639 dITP dITP
1927
['rxn20602', 'rxn26742', 'rxn26743', 'rxn26744', 'rxn26745', 'rxn26746', 'rxn26750', 'rxn26762', 'rxn26764', 'rxn26813', 'rxn26814', 'rxn26818', 'rxn26832', 'rxn26860', 'rxn26861', 'rxn26862', 'rxn26863', 'rxn26866', 'rxn26867', 'rxn26869', 'rxn26870', 'rxn26871', 'rxn26872', 'rxn26873', 'rxn26874', 'rxn26877', 'rxn26878', 'rxn26880', 'rxn26881', 'rxn29231', 'rxn25974', 'rxn25976', 'rxn25989', 'rxn25991', 'rxn26011', 'rxn26013', 'rxn26026', 'rxn26028', 'rxn26075', 'rxn26081', 'rxn26083', 'rxn26092', 'rxn26096', 'rxn26098', 'rxn26118', 'rxn26

In [17]:
co2_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0456' in tmp and tmp['s_0456']<0:
            co2_reaction.append(reaction.id)

print(len(co2_reaction))
print(co2_reaction)

183
['rxn22365', 'rxn22366', 'rxn22367', 'rxn22368', 'rxn22369', 'rxn22370', 'rxn22371', 'rxn22372', 'rxn22373', 'rxn18291', 'rxn18292', 'rxn19478', 'rxn19479', 'rxn19480', 'rxn19481', 'rxn19482', 'rxn19483', 'rxn19488', 'rxn19489', 'rxn19490', 'rxn19491', 'rxn19492', 'rxn19493', 'rxn33684', 'rxn24542', 'rxn24543', 'rxn22374', 'rxn22392', 'rxn22394', 'rxn22395', 'rxn22396', 'rxn22397', 'rxn22398', 'rxn22399', 'rxn22400', 'rxn22401', 'rxn22402', 'rxn22403', 'rxn22404', 'rxn22627', 'rxn22628', 'rxn22629', 'rxn22634', 'rxn22635', 'rxn22636', 'rxn30031', 'rxn30032', 'rxn30035', 'rxn30036', 'rxn30039', 'rxn30040', 'rxn30043', 'rxn30047', 'rxn30048', 'rxn30051', 'rxn30052', 'rxn30055', 'rxn30056', 'rxn30059', 'rxn30060', 'rxn30063', 'rxn30064', 'rxn30067', 'rxn30068', 'rxn15074', 'rxn15075', 'rxn15076', 'rxn17075', 'rxn17720', 'rxn38142', 'rxn38705', 'rxn36296', 'rxn42125', 'rxn42584', 'rxn12749', 'rxn12754', 'rxn13442', 'rxn13444', 'rxn16655', 'rxn16877', 'rxn16878', 'rxn16879', 'rxn16880',

In [18]:
o2_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1275' in tmp and tmp['s_1275']>0:
            o2_reaction.append(reaction.id)

print(len(o2_reaction))
print(o2_reaction)

318
['rxn19747', 'rxn20546', 'rxn20889', 'rxn20893', 'rxn20922', 'rxn26587', 'rxn26588', 'rxn26589', 'rxn27630', 'rxn27631', 'rxn27632', 'rxn27633', 'rxn27635', 'rxn27637', 'rxn27639', 'rxn28148', 'rxn28338', 'rxn28448', 'rxn29020', 'rxn29021', 'rxn29022', 'rxn29023', 'rxn25397', 'rxn25403', 'rxn25435', 'rxn25436', 'rxn21499', 'rxn22021', 'rxn22343', 'rxn22344', 'rxn22345', 'rxn22346', 'rxn22365', 'rxn22366', 'rxn22367', 'rxn22368', 'rxn22369', 'rxn22370', 'rxn22371', 'rxn22372', 'rxn22373', 'rxn18955', 'rxn19478', 'rxn19479', 'rxn19480', 'rxn19481', 'rxn19482', 'rxn19483', 'rxn19488', 'rxn19489', 'rxn19490', 'rxn19491', 'rxn19492', 'rxn19493', 'rxn33892', 'rxn34028', 'rxn23857', 'rxn23858', 'rxn23859', 'rxn23881', 'rxn25099', 'rxn22374', 'rxn22392', 'rxn22394', 'rxn22395', 'rxn22396', 'rxn22397', 'rxn22398', 'rxn22399', 'rxn22400', 'rxn22401', 'rxn22402', 'rxn22403', 'rxn22404', 'rxn22558', 'rxn22560', 'rxn22562', 'rxn22563', 'rxn23273', 'rxn23350', 'rxn23388', 'rxn23619', 'rxn23624',

In [19]:
nadph_reaction = []
# nadph nadh fadh2

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1212' in tmp and tmp['s_1212']>0:  #NADPH
            nadph_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1203' in tmp and tmp['s_1203']>0:   #NADH
            nadph_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0689' in tmp and tmp['s_0689']>0:  #FADH2
            nadph_reaction.append(reaction.id)

print(len(nadph_reaction))
print(nadph_reaction)


506
['rxn20547', 'rxn20889', 'rxn20893', 'rxn20922', 'rxn26587', 'rxn26588', 'rxn26589', 'rxn26934', 'rxn26935', 'rxn26936', 'rxn26937', 'rxn25581', 'rxn25583', 'rxn21496', 'rxn21497', 'rxn21498', 'rxn18487', 'rxn18987', 'rxn34028', 'rxn34050', 'rxn23857', 'rxn23858', 'rxn23859', 'rxn24147', 'rxn24702', 'rxn22814', 'rxn22815', 'rxn22816', 'rxn22817', 'rxn22818', 'rxn22819', 'rxn22820', 'rxn22821', 'rxn23619', 'rxn23624', 'rxn29745', 'rxn32910', 'rxn32911', 'rxn15240', 'rxn16977', 'rxn16978', 'rxn17047', 'rxn37771', 'rxn37186', 'rxn40597', 'rxn42114', 'rxn13382', 'rxn13585', 'rxn13586', 'rxn13587', 'rxn13589', 'rxn13590', 'rxn13591', 'rxn13598', 'rxn13599', 'rxn13601', 'rxn13602', 'rxn13603', 'rxn13609', 'rxn13610', 'rxn13611', 'rxn13613', 'rxn13614', 'rxn13615', 'rxn13621', 'rxn13622', 'rxn13623', 'rxn13625', 'rxn13626', 'rxn13627', 'rxn13633', 'rxn13634', 'rxn13635', 'rxn13637', 'rxn13638', 'rxn13639', 'rxn13645', 'rxn13646', 'rxn13647', 'rxn13649', 'rxn13650', 'rxn13651', 'rxn13657',

In [20]:
# ['s_1360','s_0075','s_4092','s_1464'] 

yeast8_atp = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1360' in tmp and tmp['s_1360']>0:   #phosphoenolpyruvate
            yeast8_atp.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0075' in tmp and tmp['s_0075']>0:   #1,3-bisphospho-D-glycerate
            yeast8_atp.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_4092' in tmp and tmp['s_4092']>0:  #N(omega)-phospho-L-arginine
            yeast8_atp.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_1464' in tmp and tmp['s_1464']>0:  #succinyl-CoA
            yeast8_atp.append(reaction.id)


print(len(yeast8_atp))
print(yeast8_atp)

92
['rxn15221', 'rxn15222', 'rxn15223', 'rxn15225', 'rxn15227', 'rxn15228', 'rxn37124', 'rxn37125', 'rxn37126', 'rxn37127', 'rxn37128', 'rxn37129', 'rxn37132', 'rxn37133', 'rxn40008', 'rxn11167', 'rxn11168', 'rxn11169', 'rxn11170', 'rxn11171', 'rxn11174', 'rxn11175', 'rxn11176', 'rxn11177', 'rxn11178', 'rxn11179', 'rxn11180', 'rxn11181', 'rxn11183', 'rxn11184', 'rxn11187', 'rxn11189', 'rxn3317', 'rxn3318', 'rxn3319', 'rxn3320', 'rxn3321', 'rxn3322', 'rxn3323', 'rxn3324', 'rxn3325', 'rxn3326', 'rxn3327', 'rxn33221', 'rxn33239', 'rxn33259', 'rxn33277', 'rxn42377', 'rxn58640', 'rxn58658', 'rxn4940', 'rxn4958', 'rxn4976', 'rxn4994', 'rxn5018', 'rxn5036', 'rxn5054', 'rxn5072', 'rxn5092', 'rxn5110', 'rxn1395', 'rxn1396', 'rxn1459', 'rxn1460', 'rxn1484', 'rxn1532', 'rxn1563', 'rxn1564', 'rxn1315', 'rxn1316', 'rxn1332', 'rxn1348', 'rxn1364', 'rxn32224', 'rxn32225', 'rxn32226', 'rxn32227', 'rxn32228', 'rxn32229', 'rxn32230', 'rxn32231', 'rxn32232', 'rxn32233', 'rxn32234', 'rxn32235', 'rxn32236'

In [21]:
else_reaction = []

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0764' in tmp and tmp['s_0764']>0:     #glycerol
            else_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0362' in tmp and tmp['s_0362']<0:   #acetate
            else_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0140' in tmp and tmp['s_0140']>0:  #2,3-bisphospho-D-glyceric acid
            else_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0979' in tmp and tmp['s_0979']<0:  #L-citrulline
            else_reaction.append(reaction.id)

for reaction in yeast8U_mod.reactions:
    if 'rxn' in reaction.id:
        tmp = {str(k.id):v for k,v in reaction.metabolites.items()}
        if 's_0373' in tmp and tmp['s_0373']>0:  #acetyl-CoA
            else_reaction.append(reaction.id)

print(len(else_reaction))
print(else_reaction)

227
['rxn26590', 'rxn23330', 'rxn30426', 'rxn35425', 'rxn35429', 'rxn49872', 'rxn49876', 'rxn49881', 'rxn61384', 'rxn5856', 'rxn4468', 'rxn4769', 'rxn4787', 'rxn5509', 'rxn5525', 'rxn5541', 'rxn5557', 'rxn5573', 'rxn5589', 'rxn5605', 'rxn5621', 'rxn5637', 'rxn5653', 'rxn3366', 'rxn4458', 'rxn1389', 'rxn1390', 'rxn1453', 'rxn1454', 'rxn1479', 'rxn1527', 'rxn1557', 'rxn1558', 'rxn1309', 'rxn1310', 'rxn1329', 'rxn1345', 'rxn1361', 'rxn20834', 'rxn29265', 'rxn29266', 'rxn25752', 'rxn22371', 'rxn18382', 'rxn18848', 'rxn18854', 'rxn18855', 'rxn18892', 'rxn18893', 'rxn19480', 'rxn19490', 'rxn33608', 'rxn33610', 'rxn33611', 'rxn33612', 'rxn34426', 'rxn24443', 'rxn22384', 'rxn23006', 'rxn23015', 'rxn30055', 'rxn30056', 'rxn30417', 'rxn14488', 'rxn14490', 'rxn14492', 'rxn14494', 'rxn14496', 'rxn14498', 'rxn14500', 'rxn14502', 'rxn14504', 'rxn14506', 'rxn14508', 'rxn14510', 'rxn14512', 'rxn14514', 'rxn14516', 'rxn14518', 'rxn14520', 'rxn14522', 'rxn14524', 'rxn14526', 'rxn14528', 'rxn14530', 'rxn

# Calculate growth after preliminary screening

In [22]:
reaction_id = []
for i in yeast8U_mod.reactions:
    if 'rxn' in i.id:
        i.bounds = (0,0)
        reaction_id.append(i.id)

reaction_id = [x for x in reaction_id if x not in biomass_up_reaction]
reaction_id = [x for x in reaction_id if x not in atp_reaction]
reaction_id = [x for x in reaction_id if x not in atp_sim_reaction]
reaction_id = [x for x in reaction_id if x not in co2_reaction]
reaction_id = [x for x in reaction_id if x not in o2_reaction]
reaction_id = [x for x in reaction_id if x not in nadph_reaction]
reaction_id = [x for x in reaction_id if x not in yeast8_atp]
reaction_id = [x for x in reaction_id if x not in else_reaction]


# for seed_num in range(20,40):
#     random.seed(seed_num)
#     random.shuffle(reaction_id)
#     with yeast8U_mod:
#         error_reaction_4 = []
#         for i in tqdm(reaction_id):
#             reaction = yeast8U_mod.reactions.get_by_id(i)
#             if 'rxn' in reaction.id:
#                 reaction.bounds = (0,1000)
#                 yeast8U_mod.solver = 'gurobi'
#                 yeast8U_mod.optimize()
#                 if yeast8U_mod.reactions.get_by_id('r_2111').flux > 0.1:
#                     reaction.bounds = (0,0)
#                     error_reaction_4.append(reaction.id)
#         print(seed_num,len(error_reaction_4))
#     # Store the list in a binary file
#     filename = error_reaction_path + 'error_reaction_random'+str(seed_num)+'.pkl'
#     with open(filename, 'wb') as file:
#         pickle.dump(error_reaction_4, file)

In [23]:
def process_seed(args):
    seed_num, reaction_id, yeast8U_mod, error_reaction_path = args
    random.seed(seed_num)
    random.shuffle(reaction_id)
    with yeast8U_mod:
        error_reaction_4 = []
        for i in tqdm(reaction_id):
            reaction = yeast8U_mod.reactions.get_by_id(i)
            if 'rxn' in reaction.id:
                reaction.bounds = (0, 1000)
                yeast8U_mod.solver = 'gurobi'
                yeast8U_mod.optimize()
                if yeast8U_mod.reactions.get_by_id('r_2111').flux > 0.09:
                    reaction.bounds = (0, 0)
                    error_reaction_4.append(reaction.id)
        print(seed_num, len(error_reaction_4))
    # Store the list in a binary file
    filename = error_reaction_path + 'error_reaction_random' + str(seed_num) + '.pkl'
    if not os.path.exists(error_reaction_path):
        os.makedirs(error_reaction_path)
    with open(filename, 'wb') as file:
        pickle.dump(error_reaction_4, file)



if __name__ == '__main__':
    with Pool(20) as pool:
        args = [(seed_num, reaction_id, yeast8U_mod, error_reaction_path) for seed_num in range(20, 40)]
        results = list(tqdm(pool.imap(process_seed, args), total=len(args)))

  0%|          | 0/20 [00:00<?, ?it/s]

Read LP format model from file /tmp/tmp3uirc0_o.lp
Reading time = 0.05 seconds
: 3224 rows, 21336 columns, 95802 nonzeros
Read LP format model from file /tmp/tmps65gxkeo.lp
Reading time = 0.05 seconds
: 3224 rows, 21336 columns, 95802 nonzeros
Read LP format model from file /tmp/tmpiimuxg3z.lp
Reading time = 0.05 seconds
: 3224 rows, 21336 columns, 95802 nonzeros
Read LP format model from file /tmp/tmpax_7wqla.lp
Reading time = 0.05 seconds
: 3224 rows, 21336 columns, 95802 nonzeros
Read LP format model from file /tmp/tmppjhgybc7.lp
Reading time = 0.04 seconds
: 3224 rows, 21336 columns, 95802 nonzeros
Read LP format model from file /tmp/tmpr2qbh5z7.lp
Reading time = 0.04 seconds
: 3224 rows, 21336 columns, 95802 nonzeros
Read LP format model from file /tmp/tmpls09dlk9.lp
Reading time = 0.04 seconds
: 3224 rows, 21336 columns, 95802 nonzeros
Read LP format model from file /tmp/tmprjgryrr0.lp
Reading time = 0.05 seconds
: 3224 rows, 21336 columns, 95802 nonzeros
Read LP format model fro

100%|██████████| 3480/3480 [06:50<00:00,  8.48it/s]


20 74


100%|██████████| 3480/3480 [06:51<00:00,  8.46it/s]


...

100%|██████████| 3480/3480 [09:02<00:00,  6.42it/s]


21 224


100%|██████████| 3480/3480 [10:04<00:00,  5.75it/s]


24 372


100%|██████████| 20/20 [24:26<00:00, 73.31s/it]   


# Select and delete the pkl file with the fewest reactions

In [24]:
# error_reaction_path = "../../Results/not_lipid/top50_0.3/error_reaction_random/"
files = os.listdir(error_reaction_path)

# Find the file with the minimum error_reaction
min_error_reaction = 1000
min_error_reaction_file = None

for file in files:
    file_path = os.path.join(error_reaction_path, file)
    with open(file_path, 'rb') as f:
        error_reaction_data = pickle.load(f)
        if len(error_reaction_data) < min_error_reaction:
            min_error_reaction = len(error_reaction_data)
            min_error_reaction_file = file  # Store the file name

print(min_error_reaction_file)


file_path = os.path.join(error_reaction_path, min_error_reaction_file)
with open(file_path, 'rb') as f:
    error_reaction_data = pickle.load(f)
print(f"Loaded data length: {len(error_reaction_data)}")


error_reaction_random30.pkl
Loaded data length: 73


In [25]:
error_reaction_data

['rxn1438',
 'rxn2698',
 'rxn9444',
 'rxn10078',
 'rxn1375',
 'rxn1196',
 'rxn24806',
 'rxn20876',
 'rxn9056',
 'rxn1366',
 'rxn34652',
 'rxn8936',
 'rxn2582',
 'rxn5860',
 'rxn1380',
 'rxn52644',
 'rxn1365',
 'rxn6330',
 'rxn401',
 'rxn10077',
 'rxn2701',
 'rxn7733',
 'rxn1443',
 'rxn7092',
 'rxn9282',
 'rxn1430',
 'rxn13266',
 'rxn21104',
 'rxn7333',
 'rxn1376',
 'rxn4280',
 'rxn22800',
 'rxn24813',
 'rxn10958',
 'rxn4261',
 'rxn31636',
 'rxn5169',
 'rxn1386',
 'rxn31996',
 'rxn23331',
 'rxn49028',
 'rxn40253',
 'rxn6360',
 'rxn62571',
 'rxn7481',
 'rxn7030',
 'rxn7732',
 'rxn13820',
 'rxn27263',
 'rxn45306',
 'rxn24804',
 'rxn4779',
 'rxn1379',
 'rxn1671',
 'rxn8956',
 'rxn2576',
 'rxn679',
 'rxn1374',
 'rxn9281',
 'rxn1385',
 'rxn3312',
 'rxn4774',
 'rxn24812',
 'rxn1439',
 'rxn7336',
 'rxn24793',
 'rxn8372',
 'rxn1373',
 'rxn3362',
 'rxn1450',
 'rxn3191',
 'rxn1253',
 'rxn1180']

In [26]:
del_reaction_lst = biomass_up_reaction + atp_reaction + atp_sim_reaction + co2_reaction + o2_reaction + nadph_reaction + yeast8_atp + else_reaction + error_reaction_data
print(len(del_reaction_lst))
del_reaction_lst = list(set(del_reaction_lst))
print(len(del_reaction_lst))

3489
3130


In [27]:
del_reaction_lst

['rxn39117',
 'rxn33221',
 'rxn44271',
 'rxn53541',
 'rxn16880',
 'rxn61147',
 'rxn8960',
 'rxn53589',
 'rxn4937',
 'rxn3095',
 'rxn22396',
 'rxn2701',
 'rxn46760',
 'rxn7890',
 'rxn3131',
 'rxn23122',
 'rxn22395',
 'rxn23781',
 'rxn23714',
 'rxn13686',
 'rxn8427',
 'rxn31428',
 'rxn31326',
 'rxn57529',
 'rxn5349',
 'rxn1396',
 'rxn56392',
 'rxn23688',
 'rxn6077',
 'rxn13447',
 'rxn31658',
 'rxn35851',
 'rxn6421',
 'rxn2018',
 'rxn58659',
 'rxn8945',
 'rxn5697',
 'rxn23722',
 'rxn16125',
 'rxn2983',
 'rxn23782',
 'rxn6398',
 'rxn4399',
 'rxn40253',
 'rxn3034',
 'rxn6267',
 'rxn5416',
 'rxn3024',
 'rxn52103',
 'rxn97',
 'rxn49587',
 'rxn60663',
 'rxn69530',
 'rxn5962',
 'rxn4769',
 'rxn18183',
 'rxn9946',
 'rxn7823',
 'rxn45313',
 'rxn3711',
 'rxn31761',
 'rxn44195',
 'rxn46744',
 'rxn50878',
 'rxn49579',
 'rxn5050',
 'rxn23661',
 'rxn5089',
 'rxn3736',
 'rxn12963',
 'rxn14388',
 'rxn13590',
 'rxn13639',
 'rxn4488',
 'rxn22344',
 'rxn23732',
 'rxn18211',
 'rxn33010',
 'rxn6430',
 'rxn64

In [28]:
# filename = '../Data/analysis/error_reaction_all.pkl'
with open(error_reaction_all_path, 'wb') as file:
    pickle.dump(del_reaction_lst, file)

In [29]:
# with open(error_reaction_all_path, 'rb') as file:
#     del_reaction_lst_ = pickle.load(file)
# del_reaction_lst_

In [30]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_path)
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model = rxndb_total_info_to_model[~rxndb_total_info_to_model['NO'].isin(del_reaction_lst)]
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_del_reaction_path, index=False)

(7889, 8)
(4759, 8)


In [31]:
get_yeast8U(metabolites_info_to_GEM_path,rxndb_total_info_to_model_del_reaction_path,yeast8_reaction_in_rxndb_json,yeast870_path,yeast8U_del_path)

(4151, 4)
(4759, 8)
(4759, 8)
(3407, 8)


100%|██████████| 3407/3407 [00:13<00:00, 254.81it/s]


In [32]:
yeast8U_del = cobra.io.load_yaml_model(yeast8U_del_path)
yeast8U_del.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.6434,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.04552,0,0.00%
s_0925,r_1861,2.736E-06,0,0.00%
s_1277,r_1992,2.2,0,0.00%
s_1324,r_2005,0.04504,0,0.00%
s_1374,r_2020,0.0003267,0,0.00%
s_1438,r_2049,0.0003572,0,0.00%
s_1468,r_2060,0.00774,0,0.00%
s_4200,r_4593,0.0001161,0,0.00%


In [33]:
# for reaction_id in tqdm(del_reaction_lst):
#     reaction = yeast8U_mod.reactions.get_by_id(reaction_id)
#     yeast8U_mod.remove_reactions([reaction])

In [34]:
# yeast8U_del_path = '../Data/model/yeast8U_Final_mod_del.yml'

# cobra.io.save_yaml_model(yeast8U_mod,yeast8U_del_path)